# Actor-Critic


## 1 env & utils

In [7]:
import gym
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
from torch.optim import Adam
from torch.autograd import Variable
import matplotlib.pyplot as plt

torch.manual_seed(28)
%matplotlib inline

In [8]:
EPOCHS = 1000
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
env = gym.make("CartPole-v1")    # 0 up, 1 right, 2 down, 3 left
env.reset()
env.action_space.n

2

In [9]:
def move_avg(a,n=100,mode="valid"):	
    return(np.convolve(a, np.ones((n,))/n, mode=mode))	
    
def plot_reward(model_name, rew): 
    plt.figure(figsize=[15, 6])
    
    plt.subplot(1,2,1)
    X = np.linspace(1,len(rew),len(rew))
    plt.plot(X, rew)
    plt.xlabel('epochs')
    plt.title(model_name)

    plt.subplot(1,2,2)
    rew = move_avg(rew)
    X = np.linspace(1,len(rew),len(rew))
    plt.plot(X, rew)
    plt.xlabel('epochs')
    plt.title(model_name)
    plt.show()

In [10]:
def train(agent):
    # train step
    rew_total = []
    for epoch in range(EPOCHS):
        obs = env.reset()[0]
        done = False
        final_reward = 0
        while not done:
            
            # env.render()
            loss = 0
            v, a, a_logprob = agent.net(Variable(torch.FloatTensor(obs)).to(device))
            obs_next, reward, done, _, info = env.step(a)
            x, x_dot, theta, theta_dot = obs_next

            r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
            r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
            reward = r1 + r2
                
            agent.add(v, a_logprob, reward)
            final_reward += reward

            # update network
            if done:
                loss = agent.learn()
                print('Ep: ', epoch,' | reward:%.3f'%final_reward, ' | loss:%.4f'%loss)

                rew_total.append(final_reward)
            
            obs = obs_next
    return rew_total

## 2 Algorithm

Based on PolicyGradient, actor-critic was born. 

This is essentially a method that combines Policy Gradient (Actor) and Function Approximation (Critic). The Actor selects behaviors based on probability, the Critic evaluates the score of the behavior based on the Actor's behavior, and the Actor modifies the probability of selecting the behavior based on the Critic's score.

Advantages: It can be updated in a single step, and the training efficiency is faster than Policy Gradient.

Disadvantages: It depends on the value judgment of the Critic, but it is difficult for the Critic to converge. Coupled with the update of the Actor, it is even more difficult to converge.

In [11]:
from AC import ActorCritic
agent = ActorCritic(device=device)

In [12]:
rew_total = train(agent)
plot_reward('Actor-Critic_reward', rew_total)

/root/miniconda3/envs/rl/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


ValueError: too many values to unpack (expected 4)